In [13]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pandas as pd
import time

## Data Scrapping

### NASA Mars News

In [2]:
# use splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Visit url 
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [37]:
# Create BeautifulSoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=51268): Max retries exceeded with url: /session/fb59f4fb2174baed5ee694f9e58049b3/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000239CC37BBE0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
# Find the title of the latest article
step1 = soup.select_one('ul.item_list li.slide')
news_title = step1.find('div', class_='content_title').get_text()
news_title

In [8]:
 # Find the teaser of the latest article
news_p = step1.find('div', class_='article_teaser_body').get_text()
news_p

'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'

In [9]:
# Close the browser after scraping
browser.quit()

### JPL Mars Space Images

In [14]:
# use splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Visit url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Navigate to desired page
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

browser.click_link_by_partial_text('more info')
time.sleep(5)

In [24]:
# Create BeautifulSoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [26]:
# Find and extract the current featured image
step1 = soup.find('figure', class_='lede')
img = step1.find('a')['href']
img


'/spaceimages/images/largesize/PIA16613_hires.jpg'

In [28]:
# Create url of the image
featured_image_url = 'https://www.jpl.nasa.gov' + img
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16613_hires.jpg'

In [29]:
# Close the browser after scraping
browser.quit()

### Mars Facts

In [13]:
# Scrape tabular data from webpage
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [14]:
# Convert table into dataframe
df = tables[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [15]:
# Read dataframe into html and export codes to file
df.to_html("output.html", index=False, header=False)

### Mars Hemispheres

In [31]:
# use splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [32]:
# Visit url
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [34]:
# Create list to store scrapped data
hemisphere_img_urls = []

# Create list of hemispheres
hemispheres = ['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']

# Create for loop to scrape data from each of the urls
for h in hemispheres:
    
    # Navigate to each hemisphere's webpage
    browser.click_link_by_partial_text(h)
    
    # Create BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find hemisphere image
    img = soup.find('img', class_='wide-image')['src']
    img_url = f'https://astrogeology.usgs.gov{img}'
    
    # Find title of image
    title = soup.find('div', class_='content').h2.text
    
    # Store data in a dictionary
    hemisphere_img_urls.append({
    'title': title,
    'img_url': img_url
    })
    
hemisphere_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Unenhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/128cba41ce2abd401c82c6a55d3b466a_schiaparelli_unenhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Unenhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/55f04ff759b242bdff8833374544b1be_syrtis_major_unenhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Unenhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/5277a23c6d8a3a325985965390cb77a2_valles_marineris_unenhanced.tif_full.jpg'}]

In [35]:
# Close the browser after scraping
browser.quit()